## Get embeddings for each report from OpenAI

##### Will need orbit API

In [2]:
import os, time, pandas as pd
from openai import OpenAI
import tiktoken

In [3]:
df = pd.read_csv('combined_train.csv')
df

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,TICKER,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,LULU,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,LULU,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,LULU,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,LULU,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,LULU,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,MCO,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other
4242,o_fwe9aJYEVTFtxduTAdK6sd,GE VERNOVA INC,US36828A1016,2024-12-10,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,2024-12,2024-12-31,36828A10,...,GEV,Industrials,NaN,-0.803,11.419023,0.218221,BP6H4Y1,US36828A1016,-0.014815,Event/Other
4243,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,BAC,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings
4244,o_gdd49aPyFdkFEH34pnybsJ,CITIGROUP INC,US1729674242,2024-06-18,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,2024-06,2024-06-28,17296742,...,C,Financials,40.8893,3.993,11.673367,1.276030,2297907,US1729674242,0.018456,Event/Other


In [4]:
TRACKER_FILE = "embedding_file.txt"
# Load tracker
processed = set()
if os.path.exists(TRACKER_FILE):
    with open(TRACKER_FILE) as f:
        processed = set(x.strip() for x in f)

In [5]:
# ----------------- CONFIG -----------------
client = OpenAI(api_key="") # insert your API key
EMBED_MODEL = "text-embedding-3-small"

In [6]:
# ================== CONFIG ==================
BATCH_SIZE = 400
MAX_TOKENS = 5000   # safe margin (well below 8191)

# Init tokenizer for the embedding model
encoding = tiktoken.encoding_for_model(EMBED_MODEL)

# ================== HELPERS ==================
def split_long_sentence(sentence, max_tokens=MAX_TOKENS):
    """Split one long sentence into smaller chunks based on tokens."""
    tokens = encoding.encode(sentence)
    if len(tokens) <= max_tokens:
        return [sentence]

    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = encoding.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

def embed_sentences(sentences, pages, out_path):
    all_rows = []
    expanded_sents, expanded_pages = [], []

    # Expand long sentences into multiple chunks
    for s, p in zip(sentences, pages):
        chunks = split_long_sentence(s, MAX_TOKENS)
        expanded_sents.extend(chunks)
        expanded_pages.extend([p] * len(chunks))

    # Embed in batches
    for i in range(0, len(expanded_sents), BATCH_SIZE):
        batch_sents = expanded_sents[i:i + BATCH_SIZE]
        batch_pages = expanded_pages[i:i + BATCH_SIZE]

        response = client.embeddings.create(
            model=EMBED_MODEL,
            input=batch_sents
        )
        batch_embs = [item.embedding for item in response.data]

        for pg, sent, emb in zip(batch_pages, batch_sents, batch_embs):
            all_rows.append({"page": pg, "sentence": sent, "embedding": emb})

    pd.DataFrame(all_rows).to_parquet(out_path, index=False)

# ================== MAIN LOOP ==================
files_to_process = df["pages_path"].tolist()

for idx, path in enumerate(files_to_process):
    if path in processed:
        print(f"⏩ Skipping {path}, already processed")
        continue

    try:
        t0 = time.time()
        print(f"Processing {path} ...")

        file_df = pd.read_json(path, lines=True)
        if "sentence" not in file_df:
            print(f"⚠️ No sentence col in {path}, skipping")
            continue

        sentences = file_df["sentence"].fillna("").astype(str).tolist()
        pages = file_df["page"].tolist()

        # Save embeddings
        out_path = path.replace(".txt", "_emb.parquet")
        embed_sentences(sentences, pages, out_path)

        # --- Add embedding_path to master df row ---
        df.loc[df["pages_path"] == path, "embedding_path"] = out_path

        # --- Track processed files ---
        with open(TRACKER_FILE, "a") as f:
            f.write(path + "\n")

        print(f"✅ Saved {out_path} in {time.time()-t0:.2f} sec")

    except Exception as e:
        print(f"❌ Error processing {path}: {e}")

⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjAhudR2mcDosUbKD5_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS94nO4peRbBEIbvje_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_PDplDokn_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVEq_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9NDnFMWH7KTPOIlox_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_cqKXOKpt_pages.txt, already processed
⏩ Skipping data/reports/LULULEMON_ATHLETICA_INC/f_gg6AP8qiabKgJXjpgJ2Xhu_pages.txt, already processed
⏩ Skipping data/reports/CBRE_GROUP_INC/f_0IO932Jd_pages.txt, already processed
⏩ Skipping data/reports/FLUOR_CORP/f_0kP15czH_pages.txt, already processed
⏩ Skipping data/reports/FLUOR_CORP/f_3txkUtqb6AnTpaYkhFNrYp_pages.txt, already processed
⏩ Skipping data/reports/FLUOR_CORP/f_7C3XBSMdYnX5wlWmwalrV6_pages.txt, already proces